In [2]:
import os
import pickle
from tweet import Reply, PostNew
from collections import defaultdict

In [3]:
ROOT_PICKLE = '../pickle/inputs/'

In [4]:
lexicon_attack = ['falso', 'Falso', 'FALSO']

**Primer resultado**

In [5]:
with open(os.path.join(ROOT_PICKLE, 'chequeado1186118160287698944.json.pkl'), 'rb') as fd:
    json = pickle.load(fd)

In [45]:
def preprocessing(json):
    replies = []
    thread = defaultdict(list)
    
    for id_str, tweet in json.items():
        
        full_text = tweet['full_text']
        reply_count = tweet['reply_count']
        created_at = tweet['created_at']
        display_text_range = tweet['display_text_range']
        conversation_id_str = tweet['conversation_id_str']
        
        if id_str == conversation_id_str:
            # POST
            post = PostNew(id_str=id_str,
                           created_at=created_at,
                           reply_count=reply_count,
                           display_text_range=display_text_range,
                           conversation_id_str=conversation_id_str,
                           full_text=full_text
                          )
        else:
            in_reply_to_status_id_str = tweet['in_reply_to_status_id_str']

            twt = Reply(created_at=created_at,
                        full_text=full_text,
                        id_str=id_str,
                        reply_count=reply_count,
                        #screen_name=screen_name,
                        display_text_range=display_text_range,
                        conversation_id_str=conversation_id_str,
                        in_reply_to_status_id_str=in_reply_to_status_id_str
                       )
            
            if in_reply_to_status_id_str == conversation_id_str:
                # REPLY
                replies.append(twt)
            else:
                # TRHEAD
                thread[in_reply_to_status_id_str].append(twt)
    
    for reply in replies:
        if reply.reply_count > 0:
            reply.replies = thread[reply.id_str]
            
    post.replies = replies
    return post

In [46]:
post = preprocessing(json)

In [47]:
post.is_controversial(lexicon_attack)

True

In [49]:
# Mostrar los tweets que atacan
for reply in post.replies:
    if reply.is_attack(lexicon_attack):
        print("R {}".format(reply.full_text))
    if reply.replies:
        for subreply in reply.replies:
            if subreply.is_attack(lexicon_attack):
                print("S {}".format(subreply.full_text))

R Tengo números que dicen lo contrario. Bajo. Bajo secuestros extorsivos. Pero no desaparecieron. Creo que Uds convenientemente usan el #EsVerdaderoPero. Sin embargo para mi aca es falso
R Los de “CHEQUEADO” son independientes.... FALSO
R cómo puede acreditar como verdadero un enunciado que contiene "prácticamente"? con este rigor dsp le ponen falso y exagerado a los demás
R Al usar en la frase el "prácticamente" creo que anula el verdadero y el falso.


In [59]:
with open(os.path.join(ROOT_PICKLE, 'example1iter.pkl'), 'wb') as fd:
    pickle.dump(post, fd)

**Segundo resultado**

In [50]:
# See https://radimrehurek.com/gensim/models/word2vec.html
from gensim.models import KeyedVectors

In [51]:
model = KeyedVectors.load_word2vec_format("../model/SBW-vectors-300-min5.bin.gz", binary=True)

In [52]:
# ACA HAY PROBLEMAS CON EL LEXICON

In [55]:
lexicon_attack_iter_2 = lexicon_attack + [word for word, d in model.most_similar(positive=["falso", "mentira"])]

In [56]:
lexicon_attack_iter_2

['falso',
 'Falso',
 'FALSO',
 'falsa',
 'mentirilla',
 'verdad',
 'fraudulencia',
 'engaño',
 'infundio',
 'mentiras',
 'andrómina',
 'patraña',
 'encubriéndola']

In [58]:
post.is_controversial(lexicon_attack_iter_2)

True

In [63]:
# Mostrar los tweets que atacan
for reply in post.replies:
    if reply.is_attack(lexicon_attack_iter_2):
        print("R {}".format(reply.full_text))
    if reply.replies:
        for subreply in reply.replies:
            if subreply.is_attack(lexicon_attack_iter_2):
                print("S {}".format(subreply.full_text))

R Tengo números que dicen lo contrario. Bajo. Bajo secuestros extorsivos. Pero no desaparecieron. Creo que Uds convenientemente usan el #EsVerdaderoPero. Sin embargo para mi aca es falso
R Los de “CHEQUEADO” son independientes.... FALSO
S No. El secuestro virtual efectivamente es una estafa (nadie fue secuestrado, ahí está el ardid o engaño, y te llaman pidiendo un rescate).
R cómo puede acreditar como verdadero un enunciado que contiene "prácticamente"? con este rigor dsp le ponen falso y exagerado a los demás
R Al usar en la frase el "prácticamente" creo que anula el verdadero y el falso.


In [64]:
with open(os.path.join(ROOT_PICKLE, 'example2iter.pkl'), 'wb') as fd:
    pickle.dump(post, fd)

**API TWITTER**

In [25]:
import tweepy

In [26]:
CONSUMER_KEY = 'tRe3ZBhnXDPBEKyTi4ifvPTC7' 
CONSUMER_SECRET = '0dmQlSimyEmaOPiDoFZTF0wYjO0LSzSdWmCP7afZkNHZF94Af5'
ACCESS_TOKEN = '421837149-YVewRWLSu8xyYNU1H4NtELUejGB8lzvT4KCfallG'
ACCESS_SECRET = 'mFsKYpvJaWvZGHRoy874Z4IF09CRP69srlVj2yjN6mH5L'

In [27]:
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [32]:
search = tweepy.Cursor(api.search, q="to:FedeFavre", sinceId = 1188249861721919489).items()